# Word Embeddings

## ¿Cómo representamos palabras, oraciones y significados en NLP?

## ¿Qué es una palabra?


![](img/words.png)

Cuando hablamos de palabras, podemos distinguir dos conceptos diferentes:

- **ocurrencia** (*token*) se refiere a una observación de una palabra en una cadena de texto. 

    Como hemos visto, en algunas lenguas es más o menos complejo identificar los límites de las palabras, pero en la mayoría de las lenguas occidentales y de nuestro entorno se utilizan espacios y otros signos de puntuación para delimitar las palabras.

- **tipo** (*type*) es la representación abstracta de una palabra. Cad **ocurrencia** pertenece a un **tipo** de palabra. Cuando contamos la frecuencia de las palabras de un *corpus* o colección de textos, lo que hacemos es contar el número de ocurrencias que tiene cada tipo.

In [ ]:
from nltk import word_tokenize

texts = [
    """No hubo sorpresa en Bruselas. 621 votos a favor, 49 en contra (los 'remainers' británicos entre ellos) y 13 abstenciones.""",
    """'The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced "no evidence" that any irregularities took place.'""",
    """環太平洋造山帯に属する小スンダ列島の西端に位置している。""",
]

for text in texts:
    print(word_tokenize(text))

In [ ]:
tweets = [
    """🎉¡#SORTEO! Gana una tostadora YummyToast Double. 🎁 
▪️Síguenos. 
▪️Comenta mencionando a 2 amigos junto a #Cecotec.
Tienes hasta el 9 de febrero para participar. El regalo se sorteará aleatoriamente entre los participantes. ¡Mucha suerte!.""",
    """we play for y’all 🏀‼️🖤 https://t.co/sd12vW93 #MambaMentality""",
]

for tweet in tweets:
    print(word_tokenize(tweet))

In [ ]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()

for text in texts:
    print(tokenizer.tokenize(text))

for tweet in tweets:
    print(tokenizer.tokenize(tweet))

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

for text in texts + tweets:
    doc = nlp(text)
    tokens = [token.text for token in doc]
    print(tokens)

Veamos qué tipo de tokenización se prefiere cuando son humanos los que segmentan las palabras: el [corpus de Brown](https://en.wikipedia.org/wiki/Brown_Corpus) en inglés, o [Ancora](http://clic.ub.edu/corpus/es) en español.

In [ ]:
from nltk.corpus import brown

brown_sents = brown.tagged_sents(categories="news")
for sentence in brown_sents[:3]:
    print([token for token, _tag in sentence])

In [ ]:
from nltk.corpus import cess_esp

ancora_sents = cess_esp.tagged_sents()
for sentence in ancora_sents[:3]:
    print([token for token, _tag in sentence])

## Representaciones discretas

A partir de aquí vamos a asumir que tenemos solucionado el proceso de tokenización e identificación de lo que es una palabra. ¿Cómo continuamos?

La manera más sencilla de representar una palabra es una cadena, es decir, como una secuencia ordenada de caracteres. Esto es cómodo, pero implica dos cosas:

- La cantidad de memoria que ocupa cada cada palabra varía en función de la longitud :-/

- Comprobar si dos palabras son idénticas es un proceso lento :-(

Otra opción alternativa consiste en representar las palabras como números enteros, de manera que a cada palabra se le asigna de manera más o menos arbitraria un número entero positivo.

- Todos las palabras ocupan la cantidad de memoria.

- Comprobar si dos cadenas contienen las mismas palabras es rápido :-)

- Estos identificadores arbitrarios no significan nada :-(

- No hay manera de relacionar palabras similares atendiendo a su identificador :-(


## Palabras como vectores distribucionales

> “You shall know a word by the company it keeps.”
> — John R. Firth (1957)
>
>“The meaning of a word is its use in the language (…) One cannot guess how a word functions. One has to look at its use, and learn from that.”
>— Ludwig Wittgenstein (1953)


La idea de que podemos analizar el uso de las palabras para deducir sus significado es una idea fundamental en semántica distribucional: la hipótesis distribucional. 

Esta idea inspira muchos algoritmos para aprender repesentaciones numéricas de las palanbras --> *word embeddings*.

In [ ]:
import numpy as np

from sklearn.datasets import fetch_20newsgroups

categories = ["comp.windows.x", "rec.sport.baseball", "sci.space", "talk.religion.misc"]
remove = ("headers", "footers", "quotes")

newsgroups_train = fetch_20newsgroups(
    subset="train", categories=categories, remove=remove
)

newsgroups_train.filenames.shape

In [ ]:
for doc in newsgroups_train.data[:3]:
    print(doc[:300])
    print("-" * 100)

for doc in newsgroups_train.data[-3:]:
    print(doc[:300])
    print("-" * 100)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

tweets_tokens = []
tweets_tokens.extend([word_tokenize(tweet) for tweet in tweets][0])

In [ ]:
label_encoder = LabelEncoder()

tokens_int = label_encoder.fit_transform(tweets_tokens)

token2int = dict(zip(tweets_tokens, tokens_int))
print(token2int)

In [ ]:
onehot_encoder = OneHotEncoder(sparse=False)

# aplanamos los tokens_int
tokens_int = tokens_int.reshape(len(tokens_int), 1)
onehot_tokens = onehot_encoder.fit_transform(tokens_int)

print(onehot_tokens)

In [ ]:
from keras.utils import to_categorical

onehot_tokens = to_categorical(tokens_int)

print(onehot_tokens)

In [ ]:
# invert first example
inverted = label_encoder.inverse_transform([np.argmax(onehot_encoded[0, :])])
print(inverted)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words="english")

vectors = vectorizer.fit_transform(
    newsgroups_train.data
).todense()  # (documents, vocab)

vectors.shape

Hemos convertido la collección de documentos en una matriz de datos donde los documentos son vectores de enteros.

![](img/vectorized-docs.png)

In [ ]:
vectors[0]

In [ ]:
vocab = np.array(vectorizer.get_feature_names())
vocab.shape

In [ ]:
print(vocab[:50])

In [ ]:
print(vocab[20000:20050])

In [ ]:
print(vocab[-50:])

## Palabras como vectores
